In [1]:
import gct

In [2]:
prefix = "K_DoesAlgSupportWeightTest"

def setup_data():
    print("Prepare data")
    a = [
        [0, 1, 1],
        [0, 2, 1],
        [1, 2, 1],

        [3, 4, 1],
        [3, 5, 1],
        [4, 5, 1],

        [6, 7, 1],
        [7, 8, 1],
        [6, 8, 1],
    ]

    if 0:
        a += [            [1, 3, 0.000001],
        [2, 4, 0.000001], ]
    else:
        for i in [0, 1, 2]:
            for j in [3, 4, 5]:
                a.append([i, j, 0.000001])        
        a.append([6, 4, 0.000001])        

    gt = {0:[0, 1, 2], 1:[3, 4, 5], 2:[6, 7, 8]}
    gt = [[v, u] for u, vv in gt.items() for v in vv]
    gct.remove_data(prefix + '_*', dry_run=False)

    data_w = gct.create_dataset(name=prefix + "_w1", edgesObj=a, groundtruthObj=gt, directed=False, weighted=True, overide=True)
    data_uw = data_w.as_unweight(newname=prefix + "_uw1", overide=True)
    return data_w,data_uw


In [3]:
data_w,data_uw=setup_data()

/opt/conda/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
2020-04-22 21:39:17,161 - Dataset:K_DoesAlgSupportWeightTest_w1 - INFO - writing/data/data/K_DoesAlgSupportWeightTest_w1/edges.parq
INFO:Dataset:K_DoesAlgSupportWeightTest_w1:writing/data/data/K_DoesAlgSupportWeightTest_w1/edges.parq


Prepare data


2020-04-22 21:39:17,572 - Clustering - INFO - Writing /data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
INFO:Clustering:Writing /data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
/opt/conda/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
2020-04-22 21:39:17,582 - Dataset:K_DoesAlgSupportWeightTest_uw1 - INFO - writing/data/data/K_DoesAlgSupportWeightTest_uw1/edges.parq
INFO:Dataset:K_DoesAlgSupportWeightTest_uw1:writing/data/data/K_DoesAlgSupportWeightTest_uw1/edges.parq


In [4]:
data_w.name,data_uw.name

('K_DoesAlgSupportWeightTest_w1', 'K_DoesAlgSupportWeightTest_uw1')

In [11]:
import os
from gct import config,utils
def has_run_1(runame, dsname):
    fpath = os.path.join(config.get_result_file_path(dsname=dsname, runname=runame), 'result.txt')
    return utils.file_exists(fpath)

def has_run(runame, dsname):
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', runame,dsname))
    return utils.file_exists(fpath)

def timeout(func, args=(), kwargs={}, timeout_duration=1, default=None):
    import signal

    class TimeoutError(Exception):
        pass

    def handler(signum, frame):
        raise TimeoutError()

    # set the timeout handler
    signal.signal(signal.SIGALRM, handler) 
    signal.alarm(timeout_duration)
    try:
        result = func(*args, **kwargs)
    except TimeoutError as exc:
        result = default
    finally:
        signal.alarm(0)

    return result


In [29]:

def run_alg(alg,dsname, check_skip=True):
    try:
        runname = alg
        if not check_skip or (check_skip and not has_run(runname, dsname)):
            print ("runing ", alg, dsname)
            if alg =='cdc_SVINET':
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123, max_iterations=1000)        
            else:
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123)
            print ("finished", alg,dsname)
        else:
            print("skip", runname,dsname)
    except:
        print("except", runname,dsname)
    return True
    

def run_alg_timeout(alg,dsname,timeout_duration=600):
    if not timeout(run_alg, (alg,dsname), timeout_duration=timeout_duration):
        print("tiemout", alg,dsname)
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', alg,dsname))
    !touch {fpath}

        


In [30]:
run_alg("oslom_lpm", data_w.name)

skip oslom_lpm K_DoesAlgSupportWeightTest_w1


True

In [19]:
run_alg_timeout("oslom_lpm", data_w.name)

skip oslom_lpm K_DoesAlgSupportWeightTest_w1


In [20]:
datasets=[data_w.name,data_uw.name]
for alg in gct.list_algorithms():
    for name in datasets:
        run_alg_timeout(alg, name)

2020-04-22 21:42:17,428 - oslom_Infohiermap - INFO - Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmpcgvd60ar/pajek.txt 1
INFO:oslom_Infohiermap:Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmpcgvd60ar/pajek.txt 1
2020-04-22 21:42:17,444 - oslom_Infohiermap - INFO - Made 3 clusters in 0.006088 seconds
INFO:oslom_Infohiermap:Made 3 clusters in 0.006088 seconds


runing  oslom_Infohiermap K_DoesAlgSupportWeightTest_w1
finished oslom_Infohiermap K_DoesAlgSupportWeightTest_w1


2020-04-22 21:42:17,555 - Dataset:K_DoesAlgSupportWeightTest_uw1 - INFO - writing edges to /data/data/K_DoesAlgSupportWeightTest_uw1/edges.txt
INFO:Dataset:K_DoesAlgSupportWeightTest_uw1:writing edges to /data/data/K_DoesAlgSupportWeightTest_uw1/edges.txt
2020-04-22 21:42:17,556 - Dataset:K_DoesAlgSupportWeightTest_uw1 - INFO - reading /data/data/K_DoesAlgSupportWeightTest_uw1/edges.parq
INFO:Dataset:K_DoesAlgSupportWeightTest_uw1:reading /data/data/K_DoesAlgSupportWeightTest_uw1/edges.parq
2020-04-22 21:42:17,562 - Dataset:K_DoesAlgSupportWeightTest_uw1 - INFO - finish writing /data/data/K_DoesAlgSupportWeightTest_uw1/edges.txt
INFO:Dataset:K_DoesAlgSupportWeightTest_uw1:finish writing /data/data/K_DoesAlgSupportWeightTest_uw1/edges.txt
2020-04-22 21:42:17,574 - oslom_Infohiermap - INFO - Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmp4mq9fm3q/pajek.txt 1
INFO:oslom_Infohiermap:Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmp4mq9fm3q/pajek.txt 1
2020-0

runing  oslom_Infohiermap K_DoesAlgSupportWeightTest_uw1
finished oslom_Infohiermap K_DoesAlgSupportWeightTest_uw1


2020-04-22 21:42:17,699 - oslom_Infomap - INFO - Running /opt/gct/submodules/OSLOM2/infomap_dir 123 /tmp/tmp_gf5k4wo/pajek.txt 1
INFO:oslom_Infomap:Running /opt/gct/submodules/OSLOM2/infomap_dir 123 /tmp/tmp_gf5k4wo/pajek.txt 1
2020-04-22 21:42:17,713 - oslom_Infomap - INFO - Made 1 clusters in 0.005361 seconds
INFO:oslom_Infomap:Made 1 clusters in 0.005361 seconds


...

runing  alg_lso_cluster K_DoesAlgSupportWeightTest_uw1
finished alg_lso_cluster K_DoesAlgSupportWeightTest_uw1


2020-04-22 21:43:09,816 - alg_streamcom - INFO - Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportWeightTest_w1/edges.txt -o output /data/data/K_DoesAlgSupportWeightTest_w1/edges.txt
INFO:alg_streamcom:Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportWeightTest_w1/edges.txt -o output /data/data/K_DoesAlgSupportWeightTest_w1/edges.txt
2020-04-22 21:43:09,824 - alg_streamcom - INFO - Made 5 clusters in 0.005946 seconds
INFO:alg_streamcom:Made 5 clusters in 0.005946 seconds


runing  alg_streamcom K_DoesAlgSupportWeightTest_w1
finished alg_streamcom K_DoesAlgSupportWeightTest_w1


2020-04-22 21:43:09,935 - alg_streamcom - INFO - Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportWeightTest_uw1/edges.txt -o output /data/data/K_DoesAlgSupportWeightTest_uw1/edges.txt
INFO:alg_streamcom:Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportWeightTest_uw1/edges.txt -o output /data/data/K_DoesAlgSupportWeightTest_uw1/edges.txt
2020-04-22 21:43:09,942 - alg_streamcom - INFO - Made 4 clusters in 0.005731 seconds
INFO:alg_streamcom:Made 4 clusters in 0.005731 seconds


runing  alg_streamcom K_DoesAlgSupportWeightTest_uw1
finished alg_streamcom K_DoesAlgSupportWeightTest_uw1


In [21]:
    results = {}
    d = gct.list_all_clustering_results(print_format=False)
    for u in d:
        if prefix + "_" in u:
            for run in d[u]:
                a = gct.load_clustering_result(u, run)
                print (u, run)
                results[u + run] = a



K_DoesAlgSupportWeightTest_w1 oslom_lpm
K_DoesAlgSupportWeightTest_w1 oslom_Infohiermap
K_DoesAlgSupportWeightTest_w1 oslom_Infomap
K_DoesAlgSupportWeightTest_w1 oslom_OSLOM
K_DoesAlgSupportWeightTest_w1 oslom_copra
K_DoesAlgSupportWeightTest_w1 oslom_louvain_method
K_DoesAlgSupportWeightTest_w1 oslom_modopt
K_DoesAlgSupportWeightTest_w1 pycabem_GANXiSw
K_DoesAlgSupportWeightTest_w1 pycabem_HiReCS
K_DoesAlgSupportWeightTest_w1 pycabem_LabelRank
K_DoesAlgSupportWeightTest_w1 cdc_CONGA
K_DoesAlgSupportWeightTest_w1 cdc_CliquePercolation
K_DoesAlgSupportWeightTest_w1 cdc_Connected_Iterative_Scan
K_DoesAlgSupportWeightTest_w1 cdc_EAGLE
K_DoesAlgSupportWeightTest_w1 cdc_FastCpm
K_DoesAlgSupportWeightTest_w1 cdc_GCE
K_DoesAlgSupportWeightTest_w1 cdc_LinkCommunities
K_DoesAlgSupportWeightTest_w1 cdc_MOSES
K_DoesAlgSupportWeightTest_w1 cdc_MSCD_AFG
K_DoesAlgSupportWeightTest_w1 cdc_MSCD_HSLSW
K_DoesAlgSupportWeightTest_w1 cdc_MSCD_LFK
K_DoesAlgSupportWeightTest_w1 cdc_MSCD_LFK2
K_DoesAlgSuppor

In [22]:
import pandas as pd
lst = [];
for alg in  gct.list_algorithms():
    for dsname in datasets:
        if dsname.startswith(prefix + '_uw'):
            a = [];columns = []
            dsname2 = dsname.replace(prefix + "_uw", prefix + "_w")
            a += [alg, dsname, dsname2]
            columns += ['alg', 'data1', 'data2']
            if dsname + alg in results and dsname2 + alg in results:
                cluster1 = gct.to_cluster(results[dsname + alg])
                cluster2 = gct.to_cluster(results[dsname2 + alg])
                compa = gct.ClusterComparator(cluster1, cluster2)
                a += [compa.sklean_nmi(), cluster1.is_overlap, cluster2.is_overlap,
                      cluster1.num_cluster, cluster2.num_cluster,
                     compa.OvpNMI()['NMImax']]
                columns += ['nmi_12', 'ovp1', 'ovp2', "#c1", "#c2", 'ovpnmi_12']

                gt = list(gct.load_local_graph(dsname).get_ground_truth().values())[0]            
                compa = gct.ClusterComparator(gt, cluster1)            
                a += [compa.sklean_nmi(), compa.OvpNMI()['NMImax']]
                columns += ['nmi_t1', 'ovpnmi_t1']

                compa = gct.ClusterComparator(gt, cluster2)            
                a += [compa.sklean_nmi(), compa.OvpNMI()['NMImax']]
                columns += ['nmi_t2', 'ovpnmi_t2']

                lst.append(a)



2020-04-22 21:43:10,109 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:10,119 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp2_dvhxv5/cluster1.cnl /tmp/tmp2_dvhxv5/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp2_dvhxv5/cluster1.cnl /tmp/tmp2_dvhxv5/cluster2.cnl > ovpnmioutput
2020-04-22 21:43:10,130 - Clustering - INFO - reading/data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
2020-04-22 21:43:10,136 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterCom

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpwm6ig704/cluster1.cnl /tmp/tmpwm6ig704/cluster2.cnl > ovpnmioutput
2020-04-22 21:43:10,352 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:10,362 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp5sg4fvzz/cluster1.cnl /tmp/tmp5sg4fvzz/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp5sg4fvzz/cluster1.cnl /tmp/tmp5sg4fvzz/cluster2.cnl > ovpnmioutput
2020-04-22 21:43:10,371 - Clustering - INFO - reading/data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
INFO:Clustering:reading/data/data/K_Doe

2020-04-22 21:43:10,574 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp62wqywm0/cluster1.cnl /tmp/tmp62wqywm0/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp62wqywm0/cluster1.cnl /tmp/tmp62wqywm0/cluster2.cnl > ovpnmioutput
2020-04-22 21:43:10,583 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:10,592 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmphdpj4krl/cluster1.cnl /tmp/tmphdpj4krl/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmphdpj4krl/cluster1.cnl /tmp/tmphdpj4krl/cluster2.

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpmqakyeng/cluster1.cnl /tmp/tmpmqakyeng/cluster2.cnl > ovpnmioutput
2020-04-22 21:43:10,815 - Clustering - INFO - reading/data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
2020-04-22 21:43:10,821 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:10,829 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpqticmqn2/cluster1.cnl /tmp/tmpqticmqn2/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpqticmqn2/cluster1.cnl /tmp

INFO:ClusterComparator:resulting 6 nodes out of 9,6
2020-04-22 21:43:11,051 - ClusterComparator - INFO - resulting 6 nodes out of 9,6
INFO:ClusterComparator:resulting 6 nodes out of 9,6
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:11,070 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:11,076 - Clustering - INFO - reading/data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
INFO:Clustering:reading/

...

2020-04-22 21:43:13,089 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:13,097 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp2q5q8u9q/cluster1.cnl /tmp/tmp2q5q8u9q/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp2q5q8u9q/cluster1.cnl /tmp/tmp2q5q8u9q/cluster2.cnl > ovpnmioutput
2020-04-22 21:43:13,111 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 

2020-04-22 21:43:13,278 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp49jej0uo/cluster1.cnl /tmp/tmp49jej0uo/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp49jej0uo/cluster1.cnl /tmp/tmp49jej0uo/cluster2.cnl > ovpnmioutput
2020-04-22 21:43:13,292 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:13,301 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpqzino0al/cluster1.cnl /tmp/tmpqzino0al/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpqzino0al/cluster1.cnl /tmp/tmpqzino0al/cluster2.

2020-04-22 21:43:13,499 - ClusterComparator - INFO - resulting 6 nodes out of 6,6
INFO:ClusterComparator:resulting 6 nodes out of 6,6
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:13,503 - Clustering - INFO - reading/data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportWeightTest_w1/gt_default.parq
2020-04-22 21:43:13,508 - ClusterComparator - INFO - resulting 6 nodes out of 9,6
INFO:ClusterComparator:resulting 6 nodes out of 9,6
2020-04-22 21:43:13,513 - ClusterComparator - INFO - resulting 6 nodes out of 9,6
INFO:ClusterComparator:resulting 6 nodes out of 9,6
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will 

2020-04-22 21:43:13,691 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpvqvzt_w_/cluster1.cnl /tmp/tmpvqvzt_w_/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpvqvzt_w_/cluster1.cnl /tmp/tmpvqvzt_w_/cluster2.cnl > ovpnmioutput
2020-04-22 21:43:13,705 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:13,714 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp1z4f22vs/cluster1.cnl /tmp/tmp1z4f22vs/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp1z4f22vs/cluster1.cnl /tmp/tmp1z4f22vs/cluster2.

2020-04-22 21:43:13,912 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
2020-04-22 21:43:13,919 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpksht2wbh/cluster1.cnl /tmp/tmpksht2wbh/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpksht2wbh/cluster1.cnl /tmp/tmpksht2wbh/cluster2.cnl > ovpnmioutput
2020-04-22 21:43:13,927 - ClusterComparator - INFO - resulting 9 nodes out of 9,9
INFO:ClusterComparator:resulting 9 nodes out of 9,9
/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 

In [23]:
rdf = pd.DataFrame(lst, columns=columns)
for alg in  gct.list_algorithms():
    if alg not in rdf['alg'].values:
        lst.append([alg] + [None] * (len(columns) - 1))
rdf = pd.DataFrame(lst, columns=columns)    
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    display (rdf)


,alg,data1,data2,nmi_12,ovp1,ovp2,#c1,#c2,ovpnmi_12,nmi_t1,ovpnmi_t1,nmi_t2,ovpnmi_t2
0,oslom_Infohiermap,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,0.761170,False,False,2.0,3.0,0.516012,0.761170,0.516012,1.000000,1.000000
1,oslom_Infomap,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,1.000000,False,False,1.0,1.0,NaN,1.500000,NaN,1.500000,NaN
2,oslom_OSLOM,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,1.000000,False,False,2.0,2.0,1.000000,0.761170,0.516012,0.761170,0.516012
3,oslom_copra,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,0.545828,True,False,3.0,3.0,0.450049,0.439462,0.450049,1.000000,1.000000
4,oslom_louvain_method,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,0.761170,False,False,2.0,3.0,0.516012,0.761170,0.516012,1.000000,1.000000
5,oslom_lpm,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,0.761170,False,False,2.0,3.0,0.516012,0.761170,0.516012,1.000000,1.000000
6,oslom_modopt,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,0.761170,False,False,2.0,3.0,0.516012,0.761170,0.516012,1.000000,1.000000
7,pycabem_GANXiSw,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,0.761170,False,False,2.0,3.0,0.516012,0.761170,0.516012,1.000000,1.000000
8,pycabem_HiReCS,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,0.761170,False,False,2.0,3.0,0.516012,0.414873,0.181069,0.558873,0.217444
9,pycabem_LabelRank,K_DoesAlgSupportWeightTest_uw1,K_DoesAlgSupportWeightTest_w1,1.000000,False,False,3.0,3.0,1.000000,0.714336,0.542430,0.714336,0.542430


In [33]:
run_alg("cdc_HDEMON", "K_DoesAlgSupportWeightTest_w1", check_skip=False)

2020-04-22 21:48:14,819 - cdc_HDEMON - INFO - seed ignored
INFO:cdc_HDEMON:seed ignored
2020-04-22 21:48:14,820 - cdc_HDEMON - INFO - Running python /opt/gct/submodules/CommunityDetectionCodes/Algorithms/2012-DEMON/HDemon.py edges.txt w ud 0.25 5
INFO:cdc_HDEMON:Running python /opt/gct/submodules/CommunityDetectionCodes/Algorithms/2012-DEMON/HDemon.py edges.txt w ud 0.25 5


runing  cdc_HDEMON K_DoesAlgSupportWeightTest_w1
except cdc_HDEMON K_DoesAlgSupportWeightTest_w1


True